In [ ]:
%pip install -U openai --quiet

### Setup environment

Preparing Friendli Personal Access Token:  

a. Sign up at [Friendli Suite](https://friendli.ai/suite).  
b. Get a [Personal Access Token (PAT)](https://friendli.ai/suite/setting/tokens)  
c. Copy the token, it will be used in the following steps.  

In [1]:
tools = [
  {
      "type": "function",
      "function": {
          "name": "get_weather",
          "parameters": {
              "type": "object",
              "properties": {
                  "location": {"type": "string"},
                  "date": {"type": "string", "format": "date"}
              },
          },
      },
  }
]

In [2]:
from datetime import datetime

today = datetime.now()
messages = [
    {"role": "system", "content": f"You are a helpful assistant. today is {today}."},
    {"role": "user", "content": "What's the weather like in Paris today?"}
]


In [3]:
from openai import OpenAI
import os

# Get your `FRIENDLI_TOKEN` from the environment
token = os.getenv("FRIENDLI_TOKEN") or "<YOUR_FRIENDLI_TOKEN>"

client = OpenAI(
    base_url = "https://api.friendli.ai/serverless/v1",
    api_key = token
)

completion = client.chat.completions.create(
  model="meta-llama-3.1-8b-instruct",
  messages=messages,
  tools=tools,
)

print(completion.choices[0].message.tool_calls)


[ChatCompletionMessageToolCall(id='call_tffgxgwsuClvIuBR6O7Uesse', function=Function(arguments='{"location": "Paris", "date": "2025-01-08"}', name='get_weather'), type='function')]


In [4]:
import json
import random

def get_weather(location: str, date: str):
    temperature = random.randint(60, 80)
    return {"temperature": temperature, "forecast": "sunny"}

tool_call = completion.choices[0].message.tool_calls[0] 

tool_response = locals()[tool_call.function.name](**json.loads(tool_call.function.arguments))
print(tool_response)


{'temperature': 73, 'forecast': 'sunny'}


In [5]:
model_response = completion.choices[0].message

# Append the response from the model
messages.append(
    {
        "role": model_response.role,
        "tool_calls": [
            tool_call.model_dump()
            for tool_call in model_response.tool_calls
        ]
    }
)

# Append the response from the tool
messages.append(
    {
        "role": "tool",
        "content": json.dumps(tool_response),
        "tool_call_id": tool_call.id
    }
)

print(json.dumps(messages, indent=2))

[
  {
    "role": "system",
    "content": "You are a helpful assistant. today is 2025-01-08 13:31:50.510071."
  },
  {
    "role": "user",
    "content": "What's the weather like in Paris today?"
  },
  {
    "role": "assistant",
    "tool_calls": [
      {
        "id": "call_tffgxgwsuClvIuBR6O7Uesse",
        "function": {
          "arguments": "{\"location\": \"Paris\", \"date\": \"2025-01-08\"}",
          "name": "get_weather"
        },
        "type": "function"
      }
    ]
  },
  {
    "role": "tool",
    "content": "{\"temperature\": 73, \"forecast\": \"sunny\"}",
    "tool_call_id": "call_tffgxgwsuClvIuBR6O7Uesse"
  }
]


In [6]:
next_completion = client.chat.completions.create(
    model="meta-llama-3.1-8b-instruct",
    messages=messages,
    tools=tools
)

print(next_completion.choices[0].message.content)


The weather in Paris today is sunny with a temperature of 73 degrees. You can expect a pleasant day to enjoy the city.
